In [1]:
# install.packages("openNLP")
# install.packages("wordcloud")
# install.packages('stringr')
# install.packages('hunspell')
# install.packages('ds4psy')
# install.packages("glue")
# install.packages('textclean')
# install.packages("tm")
# install.packages("tokenizers")
# install.packages('parsedate')
# install.packages('spacyr')
# install.packages('caret')

In [2]:
library('stringr')
library('hunspell')
library('ds4psy')
library('glue')
library('spacyr')
library('tm')
library('dplyr',  warn.conflicts = FALSE)
library('parsedate')
library("tokenizers", warn.conflicts = FALSE)


# replacing contractions with full forms
library('textclean')
library('caret')
library("e1071")

Welcome to ds4psy (v0.9.0)!

Loading required package: NLP

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate


Loading required package: lattice



In [3]:
# Cleaning dataset for further usage
# msa = Music Sentiment Analysis

# reading dataset
msa_dataset <- read.csv('./msa_dataset.csv')
msa_dataset = filter(msa_dataset, sentiment != "relaxed")

# saving unprocessed lyrics
msa_dataset['unprocessed_lyrics'] = msa_dataset['lyrics']

In [4]:
# displaying dataset
head(msa_dataset[c('id', 'title', 'all_artists','popularity','release_date','danceability','energy','key','loudness','mode','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','sentiment')], 5)

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,sentiment
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>
1,0wsXdby1T3PWLauIkGUZzg,Hula Hoop,OMI,60,2015-10-16,0.673,0.842,1,-4.666,1,0.0248,0.00e+00,0.2940,0.509,121.986,205474,4,happy
2,4lhqb6JvbHId48OUJGwymk,Hey Brother,Avicii,77,2013-01-01,0.545,0.780,7,-4.867,0,0.0309,4.64e-05,0.0828,0.458,125.014,255093,4,happy
3,1Cq23W4ZxHTY8QbP40qjEc,All I Wanna Do,Martin Jensen,52,2016-04-08,0.728,0.794,1,-3.992,1,0.1480,0.00e+00,0.0688,0.523,112.071,194360,4,happy
4,3rchFjeKZ0bJxO7EEdM3S5,Paparazzi,Kim Dracula,1,2020-12-11,0.677,0.685,4,-5.643,0,0.0739,0.00e+00,0.1070,0.177,91.008,192358,4,angry
5,1zOOl8f7qkjj0AmvlCfLyQ,The Drum,Alan Walker,70,2022-04-29,0.783,0.762,6,-4.727,1,0.1580,6.56e-05,0.0920,0.330,127.993,189288,4,happy


In [5]:
# dataset columns
column_names = colnames(msa_dataset)
column_names

[1] "X"                  "id"                 "title"             
 [4] "all_artists"        "popularity"         "release_date"      
 [7] "danceability"       "energy"             "key"               
[10] "loudness"           "mode"               "acousticness"      
[13] "instrumentalness"   "liveness"           "valence"           
[16] "tempo"              "duration_ms"        "time_signature"    
[19] "lyrics"             "sentiment"          "unprocessed_lyrics"

In [6]:
# removing X column
msa_dataset = msa_dataset[column_names[2:length(column_names)]]
names(msa_dataset)

[1] "id"                 "title"              "all_artists"       
 [4] "popularity"         "release_date"       "danceability"      
 [7] "energy"             "key"                "loudness"          
[10] "mode"               "acousticness"       "instrumentalness"  
[13] "liveness"           "valence"            "tempo"             
[16] "duration_ms"        "time_signature"     "lyrics"            
[19] "sentiment"          "unprocessed_lyrics"

In [7]:
# first 2 lyrics and sentments
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"Hula Hoop Lyrics[Intro] Oh, hey [Verse 1] Rollerskates, tan lines Hot sun clear blue skies The waves are crashing by And when she passed me by And gave a wink and smile And I was on cloud nine Lord [Pre-Chorus] The way you move your hips And lick your lips The way you dip You got me up so high (hey-ey) And girl you got that body With them curves like a Bugatti I just wanna drive, oh [Chorus] And girl you know Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Oh-oh-oh-oh [Verse 2] And gravity defied I can't believe my eyes That thing can bring the tides, hey And round and round clockwise Like a merry-go-ride I wanna go all night, yes You might also like[Pre-Chorus] The way you move your hips And lick your lips The way you dip You got me up so high, yeah (hey-ey) And girl you got that body With them curves like a Bugatti I just wanna drive, oh [Chorus] And girl you know Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh [Bridge] Oh-oh-oh, yeah Oh-oh-oh, oh Oh-oh-oh, yeah Like a hula, hula hoop, hula, hula hoop, oh [Chorus] Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh [Outro] Oh-oh-oh-oh Round and round your loving winds me up Like a hula, hula hoop, hula, hula hoop, oh8Embed",happy
2,"Hey Brother Lyrics[Verse 1] Hey, brother There's an endless road to rediscover Hey, sister Know the water's sweet but blood is thicker [Chorus] Oh, if the sky comes falling down For you, there's nothing in this world I wouldn't do [Verse 2] Hey, brother Do you still believe in one another? Hey, sister Do you still believe in love? I wonder [Chorus] Oh, if the sky comes falling down For you, there's nothing in this world I wouldn't do [Post-Chorus] What if I'm far from home? Oh brother, I will hear you call What if I lose it all? Oh sister, I will help you out Oh, if the sky comes falling down For you, there's nothing in this world I wouldn't do [Instrumental Break] [Verse 3] Hey, brother There's an endless road to rediscover Hey, sister Do you still believe in love? I wonder [Chorus] Oh, if the sky comes falling down For you, there's nothing in this world I wouldn't do [Post-Chorus] What if I'm far from home? Oh brother, I will hear you call What if I lose it all? Oh sister, I will help you out Oh, if the sky comes falling down For you, there's nothing in this world I wouldn't doYou might also like116Embed",happy


# Data Preprocessing 
##### Here, each lyric will be transformed into a format that can be used for EDA and model building.
### Steps:
1) Converting lyrics to lower case,

2) Removing lyrics divisions,

3) Correcting spellings,

4) Removing contractions,

6) Removing special characters (punctuations, periods, commas, brackets, quotes, etc) and numbers,

7) Removing extra spaces if any

8) Tokenization of words (uni-gram)

### Searching for missing values

In [8]:
sum(is.na(msa_dataset))

[1] 0

### Parsing date to give every date the same format

In [9]:
msa_dataset = transform(msa_dataset, release_date=unlist(str_split(parsedate::parse_date(release_date), " ")[1]))

Warning message in NextMethod(.Generic):
“number of items to replace is not a multiple of replacement length”


In [10]:
# replacing ’ with '
msa_dataset = transform(msa_dataset, lyrics=str_replace_all(lyrics, "’", "\'"))

### 1) Converting lyrics to lowercase

In [11]:
msa_dataset = transform(msa_dataset, lyrics=tolower(lyrics))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics[intro] oh, hey [verse 1] rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord [pre-chorus] the way you move your hips and lick your lips the way you dip you got me up so high (hey-ey) and girl you got that body with them curves like a bugatti i just wanna drive, oh [chorus] and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh [verse 2] and gravity defied i can't believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry-go-ride i wanna go all night, yes you might also like[pre-chorus] the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey-ey) and girl you got that body with them curves like a bugatti i just wanna drive, oh [chorus] and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh [bridge] oh-oh-oh, yeah oh-oh-oh, oh oh-oh-oh, yeah like a hula, hula hoop, hula, hula hoop, oh [chorus] round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh [outro] oh-oh-oh-oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics[verse 1] hey, brother there's an endless road to rediscover hey, sister know the water's sweet but blood is thicker [chorus] oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do [verse 2] hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder [chorus] oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do [post-chorus] what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do [instrumental break] [verse 3] hey, brother there's an endless road to rediscover hey, sister do you still believe in love? i wonder [chorus] oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do [post-chorus] what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't doyou might also like116embed",happy


### 2) Removing lyrics divisions,

In [12]:
msa_dataset = transform(
    msa_dataset, 
    lyrics=str_replace_all(
        string=lyrics, 
        pattern=r"(\[(.*?)\])", 
        replacement = " "
    )
)
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics oh, hey rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high (hey-ey) and girl you got that body with them curves like a bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh and gravity defied i can't believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry-go-ride i wanna go all night, yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey-ey) and girl you got that body with them curves like a bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh, yeah oh-oh-oh, oh oh-oh-oh, yeah like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics hey, brother there's an endless road to rediscover hey, sister know the water's sweet but blood is thicker oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do hey, brother there's an endless road to rediscover hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't doyou might also like116embed",happy


### 3) Correcting spellings,

In [13]:
correct_spellings = function(lyrics){
    print("correcting spellings")
    # checking spelling mistakes
    bad.words = hunspell(lyrics)

    # splitting bad words to a list of words
    bad.words = text_to_words(bad.words[1])

    # removing duplicates
    bad.words = unique(bad.words)

    # getting suggestions
    suggested.words = unlist(lapply(hunspell_suggest(unique(unlist(bad.words))), function(words) words[1]))
    
    # replacing bad words with suggestions
    new_lyrics = ""
    for (index in 1:length(bad.words)){
        print(glue(r"(\b{bad.words[index]}\b)"))
        new_lyrics = str_replace_all(
            string = lyrics, 
            pattern = glue(r"(\b{bad.words[index]}\b)"), 
            replacement = suggested.words[index]
        )
    }
    return (new_lyrics)
}

In [14]:
# applying correct_spellings to all lyrics
msa_dataset = transform(msa_dataset, lyrics=correct_spellings(lyrics))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

[1] "correcting spellings"
\bc\b
\brollerskates\b
\bey\b
\bbugatti\b


,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics oh, hey rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh and gravity defied i can't believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry-go-ride i wanna go all night, yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh, yeah oh-oh-oh, oh oh-oh-oh, yeah like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics hey, brother there's an endless road to rediscover hey, sister know the water's sweet but blood is thicker oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do hey, brother there's an endless road to rediscover hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there's nothing in this world i wouldn't do what if i'm far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there's nothing in this world i wouldn't doyou might also like116embed",happy


### 4) Removing contractions,

In [15]:
# replacing all contractions with their full forms
msa_dataset = transform(msa_dataset, lyrics=replace_contraction(lyrics))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics oh, hey rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh and gravity defied i can not believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry-go-ride i wanna go all night, yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh, yeah oh-oh-oh, oh oh-oh-oh, yeah like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics hey, brother there is an endless road to rediscover hey, sister know the water's sweet but blood is thicker oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother there is an endless road to rediscover hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not doyou might also like116embed",happy


### 6) Removing special characters (punctuations, periods, commas, brackets, quotes, etc) and numbers,

In [16]:
# replacing any internet slags that might have slipped in to the lyrics
msa_dataset = transform(msa_dataset, lyrics=replace_internet_slang(lyrics))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics oh, hey rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh and gravity defied i can not believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry-go-ride i wanna go all night, yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey-ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh, yeah oh-oh-oh, oh oh-oh-oh, yeah like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh-oh-oh-oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics hey, brother there is an endless road to rediscover hey, sister know the water's sweet but blood is thicker oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother there is an endless road to rediscover hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not doyou might also like116embed",happy


In [17]:
# replacing underscores with spaces
msa_dataset = transform(msa_dataset, lyrics=str_replace_all(lyrics, r"(\_?\-)", " "))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,"hula hoop lyrics oh, hey rollerskates, tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high (hey ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh oh oh oh and gravity defied i can not believe my eyes that thing can bring the tides, hey and round and round clockwise like a merry go ride i wanna go all night, yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high, yeah (hey ey) and girl you got that body with them curves like a Bugatti i just wanna drive, oh and girl you know round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh oh oh, yeah oh oh oh, oh oh oh oh, yeah like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh oh oh oh oh round and round your loving winds me up like a hula, hula hoop, hula, hula hoop, oh8embed",happy
2,"hey brother lyrics hey, brother there is an endless road to rediscover hey, sister know the water's sweet but blood is thicker oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother do you still believe in one another? hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not do hey, brother there is an endless road to rediscover hey, sister do you still believe in love? i wonder oh, if the sky comes falling down for you, there is nothing in this world i would not do what if I am far from home? oh brother, i will hear you call what if i lose it all? oh sister, i will help you out oh, if the sky comes falling down for you, there is nothing in this world i would not doyou might also like116embed",happy


In [18]:
msa_dataset['lyrics_with_punctuations'] = msa_dataset$lyrics

In [19]:
# removing all punctuations
msa_dataset = transform(msa_dataset, lyrics=tolower(removePunctuation(lyrics)))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,hula hoop lyrics oh hey rollerskates tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh and gravity defied i can not believe my eyes that thing can bring the tides hey and round and round clockwise like a merry go ride i wanna go all night yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high yeah hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh yeah oh oh oh oh oh oh oh yeah like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh8embed,happy
2,hey brother lyrics hey brother there is an endless road to rediscover hey sister know the waters sweet but blood is thicker oh if the sky comes falling down for you there is nothing in this world i would not do hey brother do you still believe in one another hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not do hey brother there is an endless road to rediscover hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not doyou might also like116embed,happy


In [20]:
# removing all numbers
msa_dataset = transform(msa_dataset, lyrics=str_replace_all(lyrics, r"(\d)", " "))
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,hula hoop lyrics oh hey rollerskates tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh and gravity defied i can not believe my eyes that thing can bring the tides hey and round and round clockwise like a merry go ride i wanna go all night yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high yeah hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh yeah oh oh oh oh oh oh oh yeah like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh embed,happy
2,hey brother lyrics hey brother there is an endless road to rediscover hey sister know the waters sweet but blood is thicker oh if the sky comes falling down for you there is nothing in this world i would not do hey brother do you still believe in one another hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not do hey brother there is an endless road to rediscover hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not doyou might also like embed,happy


### 7) Removing extra spaces if any

In [21]:
# replacing all whitespace characters with one space
msa_dataset = transform(
    msa_dataset, 
    lyrics=str_replace_all(
        string=lyrics, 
        pattern=r"(\s+)", 
        replacement = " "
    )
)
head(msa_dataset[c('lyrics', 'sentiment')], 2)

,lyrics,sentiment
,<chr>,<chr>
1,hula hoop lyrics oh hey rollerskates tan lines hot sun clear blue skies the waves are crashing by and when she passed me by and gave a wink and smile and i was on cloud nine lord the way you move your hips and lick your lips the way you dip you got me up so high hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh and gravity defied i can not believe my eyes that thing can bring the tides hey and round and round clockwise like a merry go ride i wanna go all night yes you might also like the way you move your hips and lick your lips the way you dip you got me up so high yeah hey ey and girl you got that body with them curves like a bugatti i just wanna drive oh and girl you know round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh yeah oh oh oh oh oh oh oh yeah like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh oh oh oh oh round and round your loving winds me up like a hula hula hoop hula hula hoop oh embed,happy
2,hey brother lyrics hey brother there is an endless road to rediscover hey sister know the waters sweet but blood is thicker oh if the sky comes falling down for you there is nothing in this world i would not do hey brother do you still believe in one another hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not do hey brother there is an endless road to rediscover hey sister do you still believe in love i wonder oh if the sky comes falling down for you there is nothing in this world i would not do what if i am far from home oh brother i will hear you call what if i lose it all oh sister i will help you out oh if the sky comes falling down for you there is nothing in this world i would not doyou might also like embed,happy


### 8) Tokenization of words (uni-gram)

In [22]:
# holds sentiments of each lyric
sentiments.classes = msa_dataset$sentiment
table(sentiments.classes)

sentiments.classes
angry happy   sad 
  271   227   341 

In [23]:
# # a list of vectors in which each vector is a tokenized lyrics
# # converting lyrics in batches of 100
# lyrics.tokenized = list()
# print(lyrics.tokenized)
# index = 1
# while (index <= length(msa_dataset$lyrics)){
#     lyrics.tokenized = append(
#         lyrics.tokenized, 
#         tokenize_words(
#             msa_dataset$lyrics[index]
#         )
#     )
#     index = index + 1
# }
    

In [24]:
# lyrics.tokenized[1]

In [25]:
length(sentiments.classes)

[1] 839

In [26]:
# length(lyrics.tokenized)

In [27]:
# # Tokenization of the lyrics for each class
# angry.tokenized = tokenize_words(split(msa_dataset, msa_dataset$sentiment)$angry$lyrics)
# happy.tokenized = tokenize_words(split(msa_dataset, msa_dataset$sentiment)$happy$lyrics)
# sad.tokenized = tokenize_words(split(msa_dataset, msa_dataset$sentiment)$sad$lyrics)
# # relaxed.tokenized = tokenize_words(split(msa_dataset, msa_dataset$sentiment)$relaxed$lyrics)
# length(angry.tokenized)
# length(happy.tokenized)
# length(sad.tokenized)
# # length(relaxed.tokenized)

### 9) Lemmatization

In [28]:
# download spacy english model in terminal
# install pip3 install protobuf==3.20.0
# python3 -m spacy download en

In [29]:
# initializing spacyr
spacy_initialize(model = 'en_core_web_sm')

Finding a python executable with spaCy installed...

spaCy (language model: en_core_web_sm) is installed in /usr/local/bin/python3

successfully initialized (spaCy Version: 3.4.3, language model: en_core_web_sm)

(python options: type = "python_executable", value = "/usr/local/bin/python3")



In [30]:
# fetching english stopwords
stopwords.list = stopwords(kind = "en")
print(stopwords.list[56:86])

# removing contractions from stop words because lyrics does not have stop words
stopwords.cleaned = replace_contraction(stopwords.list)
print(stopwords.cleaned[56:86])

 [1] "could"   "ought"   "i'm"     "you're"  "he's"    "she's"   "it's"   
 [8] "we're"   "they're" "i've"    "you've"  "we've"   "they've" "i'd"    
[15] "you'd"   "he'd"    "she'd"   "we'd"    "they'd"  "i'll"    "you'll" 
[22] "he'll"   "she'll"  "we'll"   "they'll" "isn't"   "aren't"  "wasn't" 
[29] "weren't" "hasn't"  "haven't"
 [1] "could"      "ought"      "I am"       "you are"    "he is"     
 [6] "she is"     "it is"      "we are"     "they are"   "I have"    
[11] "you have"   "we have"    "they have"  "I would"    "you would" 
[16] "he would"   "she would"  "we would"   "they would" "I will"    
[21] "you will"   "he will"    "she will"   "we will"    "they will" 
[26] "is not"     "are not"    "was not"    "were not"   "has not"   
[31] "haven't"   


In [31]:
# # adding stopwords
# stopwords.cleaned = append(stopwords.cleaned, c(
#     "oh", "ey", "ooh", "oooh", "hey", "hoop", "hula", "lyric", "yeah", "wanna", "hey", "will", "know", "like"
# ))
# stopwords.cleaned

In [32]:
# pos tagging, lematization
# Not using transform here because it combines every thing into one long string
important = c("ADJ", "ADV", "NOUN")
lemmatized_lyrics = c()
for (lyrics in msa_dataset$lyrics){
    
    features = filter(
        spacy_parse(lyrics, tag=TRUE, pos=TRUE), 
        (pos %in% important) & (entity == "") & 
        !(lemma %in% stopwords.cleaned)
    )
    
    lemmatized_lyrics = append(
        lemmatized_lyrics, 
        paste(features$lemma, collapse = ' ')
    )
}

In [33]:
lemmatized_lyrics

[1] "lyric clear sky wave wink smile cloud way hip lip way high ey girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop gravity eye thing tide round round clockwise go ride night also way hip lip way high girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop loving wind hula hula hoop hula hoop embed"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
  [2] "brother lyric brother endless road sister water sweet blood thick sky world brother still sister still love sky world far home brother sister sky world brother endless road sister still love sky world far home brother sister sky world also"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
  [3] "lyric window maybe home number now message phone stalker just close runnin time insane somethin bout mind love tight body tight body tight body picture wall even mama heart stalker just close runnin time insane somethin bout mind love tight body tight body love tight body tight body also tight body embed"         

In [34]:
# creating column for lemmatization
msa_dataset['lemmatized_lyrics'] = lemmatized_lyrics
msa_dataset$lemmatized_lyrics[1]

[1] "lyric clear sky wave wink smile cloud way hip lip way high ey girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop gravity eye thing tide round round clockwise go ride night also way hip lip way high girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop loving wind hula hula hoop hula hoop embed"

### Removing stopwords

In [35]:
# remove_stopwords = function(lyrics){
#     new_lyrics = lyrics
    
#     for (stopword in stopwords.cleaned){
#         new_lyrics = str_remove_all(tolower(new_lyrics), glue(r"(\b\s?{stopword}\b)"))
#     }
#     return (new_lyrics)
# }
# msa_dataset = transform(msa_dataset, lyrics_no_stopwords=remove_stopwords(lemmatized_lyrics))
# msa_dataset$lyrics_no_stopwords[2]

In [36]:
names(msa_dataset)

[1] "id"                       "title"                   
 [3] "all_artists"              "popularity"              
 [5] "release_date"             "danceability"            
 [7] "energy"                   "key"                     
 [9] "loudness"                 "mode"                    
[11] "acousticness"             "instrumentalness"        
[13] "liveness"                 "valence"                 
[15] "tempo"                    "duration_ms"             
[17] "time_signature"           "lyrics"                  
[19] "sentiment"                "unprocessed_lyrics"      
[21] "lyrics_with_punctuations" "lemmatized_lyrics"

In [37]:
# msa_dataset_cleaned = fread("dataset_with_no_stopwords.csv",  select= c("lyrics_no_stopwords", "sentiment"))
# msa_dataset_cleaned = msa_dataset[c("mode", "valence", "energy", "danceability", "acousticness", "lemmatized_lyrics", "sentiment")]
msa_dataset_cleaned = msa_dataset[c("lemmatized_lyrics", "sentiment")]

head(msa_dataset_cleaned, 2)

,lemmatized_lyrics,sentiment
,<chr>,<chr>
1,lyric clear sky wave wink smile cloud way hip lip way high ey girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop gravity eye thing tide round round clockwise go ride night also way hip lip way high girl body bugatti just girl round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hoop round loving wind hula hula hoop hula hula hoop loving wind hula hula hoop hula hoop embed,happy
2,brother lyric brother endless road sister water sweet blood thick sky world brother still sister still love sky world far home brother sister sky world brother endless road sister still love sky world far home brother sister sky world also,happy


### 11) Document Term Matrix

- The document term matrix is a technique of structuring text data in such a away that documents will be placed in the row and the terms of each document in the rows of the dataframe. The value of each term will be either zero or one representing the absence or presence of the term in a particular document, respectively. Each cell contains the weight of a term in a particular document. The weight of a term in a document can be determined using any of the following approaches;
<br>

   - **Binary weights:** Here, the weight of a term can only be 0 or 1 representing the abscence or prescence of the term in a document, respectively.
   - **Term frequency:** The weight of a term corresponse to the total number of times it appears in a specify document. Here, it is assumed that the higher the frequency of a term, the more important it is.
   - **Inverse Document Frequency:** The idea here is to associate less common terms with higher weights. This is done at the corpus level and thus describes the entire corpus and not just a document.
    <br>
    IDF(term) = 1 + log(N/DF(term))
    <br>
    where 
    <br>
    IDF = Inverse Document Frequency
    <br>
    DF = Document Frequency.
    <br>
    DF(term) = Number of documents containing term
    <br>
    
  - **Term Frequecny - Inverse Document Frequecy:** This approach seeks to quantify the importance of a term in a document amongst a collection of documents.
   <br>
   TF-IDF(term) = TF(term) * IDF(term)
   
   <br>
   <br>
   **REF**
   <br>
   https://medium.com/analytics-vidhya/tdm-term-document-matrix-and-dtm-document-term-matrix-8b07c58957e2
  

In [38]:
lyrics = msa_dataset_cleaned$lemmatized_lyrics

In [39]:
# creating a corpus of all lyrics. A corpus is simply a collection of text/documents
lyrics_corpus = Corpus(VectorSource(lyrics))
lyrics_corpus

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 839

In [40]:
# creating a document term matrix from lyrics corpus
lyrics_document_term_matrix = DocumentTermMatrix(lyrics_corpus, control = list(
    weighting=function(x) weightTfIdf(x, normalize = TRUE)
))
lyrics_document_term_matrix

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
“custom functions are ignored”


<<DocumentTermMatrix (documents: 839, terms: 5191)>>
Non-/sparse entries: 32351/4322898
Sparsity           : 99%
Maximal term length: 17
Weighting          : term frequency - inverse document frequency (normalized) (tf-idf)

In [41]:
# converting dtm to matrix
lyrics_dtm_matrix = as.matrix(lyrics_document_term_matrix)
head(lyrics_dtm_matrix, 2)

,also,body,bugatti,clear,clockwise,cloud,embed,eye,girl,gravity,⋯,unsaid,alarm,cheater,itch,halfway,amy,lime,adio,ciao,kissing
1,1.220260e-05,0.05330966,0.1152846,0.0434579,0.06888317,0.03875602,0.01206596,0.01741233,0.06557865,0.06179097,⋯,0,0,0,0,0,0,0,0,0,0
2,4.301417e-05,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,0


In [42]:
length(lyrics_dtm_matrix)

[1] 4355249

In [43]:
# combining document term matrix with their respective sentiments
msa_dataset_dtm = bind_cols(lyrics_dtm_matrix, sentiment = msa_dataset_cleaned$sentiment)
head(msa_dataset_dtm, 5)

also,body,bugatti,clear,clockwise,cloud,embed,eye,girl,gravity,⋯,alarm,cheater,itch,halfway,amy,lime,adio,ciao,kissing,sentiment
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1.220260e-05,0.05330966,0.1152846,0.0434579,0.06888317,0.03875602,0.01206596,0.01741233,0.06557865,0.06179097,⋯,0,0,0,0,0,0,0,0,0,happy
4.301417e-05,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,happy
3.308782e-05,0.57820472,0.0000000,0.0000000,0.00000000,0.00000000,0.03271730,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,happy
2.177933e-05,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,angry
2.493575e-05,0.21787424,0.0000000,0.0000000,0.00000000,0.00000000,0.02465652,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,happy


In [44]:
# colnames(msa_dataset_cleaned) = c("mode_col", "valence_col", "energy_col", "danceability_col", "acousticness_col", "lyrics_no_stopwords", "sentiment")


In [45]:
names(msa_dataset_cleaned)

[1] "lemmatized_lyrics" "sentiment"

In [46]:
# msa_dataset_dtm = bind_cols(msa_dataset_cleaned[c("mode_col", "valence_col", "energy_col", "danceability_col", "acousticness_col")], msa_dataset_dtm)

### Splitting dataset
- Here, the data is divided into two halves. One for traning our model and the other for testing/validating the peformance of our model. A greater proportion of the data goes for traning. The common proportions are 70% for traning and 30% for testing/validation.

In [47]:
# Creating a random vector of TRUEs and FALSEs to be used in splitting the dataset into train and test
# 70% of the dataset is set to TRUE for training and 30% to FALSE for testing
set.seed(1)
msa_split_dataset = sample(c(TRUE, FALSE), nrow(msa_dataset_dtm), replace = TRUE, prob = c(0.7, 0.3))

In [48]:
# train dataset
msa_train_dataset = msa_dataset_dtm[msa_split_dataset, ]
head(msa_train_dataset, 3)

also,body,bugatti,clear,clockwise,cloud,embed,eye,girl,gravity,⋯,alarm,cheater,itch,halfway,amy,lime,adio,ciao,kissing,sentiment
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1.220260e-05,0.05330966,0.1152846,0.0434579,0.06888317,0.03875602,0.01206596,0.01741233,0.06557865,0.06179097,⋯,0,0,0,0,0,0,0,0,0,happy
4.301417e-05,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,happy
3.308782e-05,0.57820472,0.0000000,0.0000000,0.00000000,0.00000000,0.03271730,0.00000000,0.00000000,0.00000000,⋯,0,0,0,0,0,0,0,0,0,happy


In [49]:
# test dataset
msa_test_dataset = msa_dataset_dtm[!msa_split_dataset, ]
head(msa_test_dataset, 3)

also,body,bugatti,clear,clockwise,cloud,embed,eye,girl,gravity,⋯,alarm,cheater,itch,halfway,amy,lime,adio,ciao,kissing,sentiment
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2.177933e-05,0,0,0.00000000,0,0,0.0000000,0.00000000,0.0000000,0,⋯,0,0,0,0,0,0,0,0,0,angry
3.308782e-05,0,0,0.00000000,0,0,0.0000000,0.00000000,0.0000000,0,⋯,0,0,0,0,0,0,0,0,0,sad
3.781465e-05,0,0,0.06733587,0,0,0.0186956,0.02697955,0.2540272,0,⋯,0,0,0,0,0,0,0,0,0,happy


In [50]:
table(msa_test_dataset$sentiment)


angry happy   sad 
   76    67   114 

In [51]:
table(msa_train_dataset$sentiment)


angry happy   sad 
  195   160   227 

In [52]:
# write.csv(msa_dataset, "dataset_with_no_stopwords.csv")

### Model Building

In [53]:
labels = names(msa_train_dataset)[1 :  length(names(msa_train_dataset)) - 1]
msa_model = naiveBayes(msa_train_dataset[labels], msa_train_dataset$sentiment)

In [54]:
pred = predict(msa_model, msa_train_dataset[labels])

In [55]:
confusionMatrix(as.factor(msa_train_dataset$sentiment), pred)

Confusion Matrix and Statistics

          Reference
Prediction angry happy sad
     angry     8   124  63
     happy     0   122  38
     sad       0   148  79

Overall Statistics
                                          
               Accuracy : 0.3591          
                 95% CI : (0.3201, 0.3996)
    No Information Rate : 0.677           
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0694          
                                          
 Mcnemar's Test P-Value : <2e-16          

Statistics by Class:

                     Class: angry Class: happy Class: sad
Sensitivity               1.00000       0.3096     0.4389
Specificity               0.67422       0.7979     0.6318
Pos Pred Value            0.04103       0.7625     0.3480
Neg Pred Value            1.00000       0.3555     0.7155
Prevalence                0.01375       0.6770     0.3093
Detection Rate            0.01375       0.2096     0.1357
Det

In [56]:
test_pred = predict(msa_model, msa_test_dataset[labels])

In [57]:
confusionMatrix(as.factor(msa_test_dataset$sentiment), test_pred)

Confusion Matrix and Statistics

          Reference
Prediction angry happy sad
     angry     3    46  27
     happy     1    48  18
     sad       0    84  30

Overall Statistics
                                          
               Accuracy : 0.3152          
                 95% CI : (0.2589, 0.3758)
    No Information Rate : 0.6926          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 8e-04           
                                          
 Mcnemar's Test P-Value : <2e-16          

Statistics by Class:

                     Class: angry Class: happy Class: sad
Sensitivity               0.75000       0.2697     0.4000
Specificity               0.71146       0.7595     0.5385
Pos Pred Value            0.03947       0.7164     0.2632
Neg Pred Value            0.99448       0.3158     0.6853
Prevalence                0.01556       0.6926     0.2918
Detection Rate            0.01167       0.1868     0.1167
Det

In [58]:
table(msa_test_dataset$sentiment)


angry happy   sad 
   76    67   114 